In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
import networkx as nx

def process_floor_plan(image_path):
    """
    Process floor plan image to detect rooms, walls, and doorways
    Returns room positions and connections for graph creation

    Parameters:
    image_path (str): Path to the floor plan image

    Returns:
    tuple: Original image, processed image, room positions dict, connections list, doorways list
    """
    # Load image
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"Could not load image from {image_path}")

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply binary thresholding to get walls
    _, binary = cv2.threshold(blurred, 180, 255, cv2.THRESH_BINARY_INV)

    # Morphological operations to enhance walls and remove furniture
    # Use a smaller kernel to preserve internal walls
    kernel = np.ones((3, 3), np.uint8)
    dilated = cv2.dilate(binary, kernel, iterations=1)
    eroded = cv2.erode(dilated, kernel, iterations=1)

    # Keep a copy of all walls including internal walls
    all_walls = eroded.copy()

    # Get the contours of the rooms - use RETR_LIST to get all contours including internal walls
    contours, hierarchy = cv2.findContours(eroded, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    # Create a mask for the walls - including internal walls
    wall_mask = np.zeros_like(gray)

    # First draw the external walls
    external_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 500]
    cv2.drawContours(wall_mask, external_contours, -1, 255, 2)

    # Then add internal walls using the eroded image
    internal_walls = cv2.Canny(all_walls, 50, 150)
    wall_mask = cv2.bitwise_or(wall_mask, internal_walls)

    # Result image for visualization
    result = cv2.cvtColor(wall_mask.copy(), cv2.COLOR_GRAY2BGR)

    # Re-detect rooms after adding internal walls
    # We need to use floodfill to identify rooms properly
    filled_rooms = np.zeros_like(gray)
    inverted_walls = cv2.bitwise_not(wall_mask)

    # Use connected components to find individual rooms
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(inverted_walls, connectivity=4)

    # Predefined room names (from your floor plan)
    room_names = ['KÖK', 'ALLRUM', 'SOVRUM', 'FRD', 'HALL', 'WC/D', 'KLK', 'UTEPLATS']

    # Dictionary to store room positions
    room_positions = {}
    detected_rooms = []

    # Start from 1 to skip the background
    for i in range(1, num_labels):
        area = stats[i, cv2.CC_STAT_AREA]
        if area > 500:  # Filter very small regions
            cX = int(centroids[i, 0])
            cY = int(centroids[i, 1])

            # Create a mask for this room
            room_mask = np.zeros_like(gray)
            room_mask[labels == i] = 255
            filled_rooms = cv2.bitwise_or(filled_rooms, room_mask)

            # Store room info for later use
            detected_rooms.append((cX, cY, area))

    # Match detected rooms with room names based on position
    # This is a simplified approach - in a real application, you might use OCR to read room names
    # For now, we'll assign names based on relative positions
    detected_rooms.sort(key=lambda x: x[2], reverse=True)  # Sort by area, largest first

    # Assign room names - we're guessing based on typical layout
    # This is where knowledge of the specific floor plan would help
    for i, (cX, cY, area) in enumerate(detected_rooms):
        if i < len(room_names):
            room_positions[room_names[i]] = (cX, cY)
            cv2.putText(result, room_names[i], (cX - 20, cY),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            cv2.circle(result, (cX, cY), 5, (0, 255, 0), -1)

    # Detect doorways between rooms
    doorways = []

    # Detect wall boundaries
    wall_edges = cv2.Canny(wall_mask, 50, 150)

    # Look for potential doorways using distance transform
    dist_transform = cv2.distanceTransform(cv2.bitwise_not(wall_mask), cv2.DIST_L2, 3)
    _, dist_thresh = cv2.threshold(dist_transform, 3, 255, cv2.THRESH_BINARY)
    dist_thresh = dist_thresh.astype(np.uint8)

    # Thin lines between rooms could be doorways
    door_candidates = cv2.bitwise_and(dist_thresh, wall_edges)

    # Dilate to connect nearby points
    door_kernel = np.ones((3, 3), np.uint8)
    dilated_edges = cv2.dilate(door_candidates, door_kernel, iterations=1)

    # Find door contours
    door_contours, _ = cv2.findContours(dilated_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter door candidates
    for door in door_contours:
        if cv2.contourArea(door) > 10 and cv2.contourArea(door) < 200:
            M = cv2.moments(door)
            if M["m00"] != 0:
                dX = int(M["m10"] / M["m00"])
                dY = int(M["m01"] / M["m00"])
                doorways.append((dX, dY))
                cv2.circle(result, (dX, dY), 3, (255, 0, 255), -1)  # Mark doorways

    # Determine room connections based on doorways
    connections = []

    for door in doorways:
        # Find the two closest rooms to this doorway
        closest_rooms = []
        for room, pos in room_positions.items():
            dist = np.sqrt((door[0] - pos[0])**2 + (door[1] - pos[1])**2)
            closest_rooms.append((room, dist))

        # Sort by distance
        closest_rooms.sort(key=lambda x: x[1])

        # Get the two closest rooms
        if len(closest_rooms) >= 2:
            room1 = closest_rooms[0][0]
            room2 = closest_rooms[1][0]

            # Add to connections if not already there
            connection = (room1, room2)
            reverse_connection = (room2, room1)

            if connection not in connections and reverse_connection not in connections:
                connections.append(connection)

    # Fallback: if no connections were found, create some based on proximity
    if not connections:
        room_list = list(room_positions.keys())
        for i in range(len(room_list)):
            for j in range(i+1, len(room_list)):
                room1 = room_list[i]
                room2 = room_list[j]
                pos1 = room_positions[room1]
                pos2 = room_positions[room2]

                # Calculate distance between rooms
                dist = np.sqrt((pos1[0] - pos2[0])**2 + (pos1[1] - pos2[1])**2)

                # If rooms are close enough, they might be connected
                if dist < 150:
                    connections.append((room1, room2))

    return img, result, room_positions, connections, doorways

def process_floor_plan_with_graph(image_path):
    """
    Process floor plan image and overlay a graph visualization on it
    """
    # Process the floor plan to get room positions and connections
    img, processed_img, room_positions, connections, doorways = process_floor_plan(image_path)

    # Make a copy for the final result
    result_img = img.copy()

    # Find the path from Bedroom to Living Room to Bathroom using BFS
    path_sovrum_to_allrum = find_path(connections, "SOVRUM", "ALLRUM")
    path_allrum_to_wcd = find_path(connections, "ALLRUM", "WC/D")

    # Combine the paths
    if path_sovrum_to_allrum and path_allrum_to_wcd:
        # Remove duplicate Living Room
        full_path = path_sovrum_to_allrum + path_allrum_to_wcd[1:]
    else:
        full_path = ["Path not found"]

    # Draw room nodes - smaller circles for better readability
    for room, pos in room_positions.items():
        if room in full_path:
            # Rooms in path - smaller red circles
            cv2.circle(result_img, pos, 8, (0, 0, 255), -1)  # Smaller filled circle
            # Add white letter label
            cv2.putText(result_img, room[0],
                       (pos[0]-3, pos[1]+3),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1)
        else:
            # Other rooms - smaller blue circles
            cv2.circle(result_img, pos, 8, (255, 0, 0), 1)  # Smaller outlined circle

    # Draw connections
    for room1, room2 in connections:
        pos1 = room_positions[room1]
        pos2 = room_positions[room2]

        # Check if this connection is part of our path
        is_in_path = False
        for i in range(len(full_path)-1):
            if (full_path[i] == room1 and full_path[i+1] == room2) or \
               (full_path[i] == room2 and full_path[i+1] == room1):
                is_in_path = True
                break

        # Find doorway between these rooms, if any
        doorway_between = None
        for door in doorways:
            # Calculate distances from door to both rooms
            dist1 = np.sqrt((door[0] - pos1[0])**2 + (door[1] - pos1[1])**2)
            dist2 = np.sqrt((door[0] - pos2[0])**2 + (door[1] - pos2[1])**2)

            # Check if this door is between these two rooms
            if dist1 < 150 and dist2 < 150:
                doorway_between = door
                break

        if is_in_path:
            # Path edges - thick red lines
            if doorway_between:
                # Draw path through doorway
                cv2.line(result_img, pos1, doorway_between, (0, 0, 255), 3)
                cv2.line(result_img, doorway_between, pos2, (0, 0, 255), 3)
                # Mark doorway
                cv2.circle(result_img, doorway_between, 5, (0, 255, 0), -1)
            else:
                # Direct path
                cv2.line(result_img, pos1, pos2, (0, 0, 255), 3)
        else:
            # Regular edges - thin blue lines
            if doorway_between:
                # Draw connection through doorway
                cv2.line(result_img, pos1, doorway_between, (255, 0, 0), 1)
                cv2.line(result_img, doorway_between, pos2, (255, 0, 0), 1)
            else:
                # Direct connection
                cv2.line(result_img, pos1, pos2, (255, 0, 0), 1)

    # Add path info at the bottom of the image
    path_label = "Path: " + " → ".join(full_path)

    # Get text size to center it
    text_size = cv2.getTextSize(path_label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)[0]
    text_x = (result_img.shape[1] - text_size[0]) // 2
    text_y = result_img.shape[0] - 20

    # Create a semi-transparent background for text
    text_bg_width = text_size[0] + 20
    text_bg_height = 25
    text_bg = np.zeros((text_bg_height, text_bg_width, 3), dtype=np.uint8)
    text_bg[:, :] = (240, 240, 240)  # Light gray background

    # Draw the text on the background
    cv2.putText(text_bg, path_label, (10, 18),
               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

    # Overlay the text background onto the main image
    alpha = 0.85  # Text background opacity

    # Ensure the text fits within the image boundaries
    if text_x < 0:
        text_x = 10
    if text_x + text_bg_width > result_img.shape[1]:
        text_x = result_img.shape[1] - text_bg_width - 10

    # Blend the text background with the main image
    for c in range(3):
        result_img[text_y-text_bg_height+5:text_y+5, text_x:text_x+text_bg_width, c] = \
            alpha * text_bg[:, :, c] + (1-alpha) * result_img[text_y-text_bg_height+5:text_y+5, text_x:text_x+text_bg_width, c]

    # Draw compact legend with smaller icons and tighter spacing
    # Create a semi-transparent background for the legend
    legend_width = 160
    legend_height = 110
    legend_bg = np.zeros((legend_height, legend_width, 3), dtype=np.uint8)
    legend_bg[:, :] = (240, 240, 240)  # Light gray background

    # Create the legend items on the background
    cv2.circle(legend_bg, (10, 15), 6, (0, 0, 255), -1)  # Rooms in path
    cv2.putText(legend_bg, "Rooms in path", (25, 18),
               cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)

    cv2.circle(legend_bg, (10, 35), 6, (255, 0, 0), 1)  # Other rooms
    cv2.putText(legend_bg, "Other rooms", (25, 38),
               cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)

    cv2.line(legend_bg, (5, 55), (15, 55), (0, 0, 255), 2)  # Path connections
    cv2.putText(legend_bg, "Path connections", (25, 58),
               cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)

    cv2.line(legend_bg, (5, 75), (15, 75), (255, 0, 0), 1)  # Other connections
    cv2.putText(legend_bg, "Other connections", (25, 78),
               cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)

    cv2.circle(legend_bg, (10, 95), 3, (0, 255, 0), -1)  # Doorways
    cv2.putText(legend_bg, "Doorways", (25, 98),
               cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)

    # Add a border to the legend
    cv2.rectangle(legend_bg, (0, 0), (legend_width-1, legend_height-1), (100, 100, 100), 1)

    # Overlay the legend onto the main image
    x_offset = 20
    y_offset = 20

    # Create a mask for the legend to blend it with the background
    alpha = 0.85  # Legend opacity

    # Blend the legend with the main image
    for c in range(3):
        result_img[y_offset:y_offset+legend_height, x_offset:x_offset+legend_width, c] = \
            alpha * legend_bg[:, :, c] + (1-alpha) * result_img[y_offset:y_offset+legend_height, x_offset:x_offset+legend_width, c]

    return result_img, processed_img, full_path

def find_path(graph, start, end):
    """
    Find a path between two rooms using BFS algorithm
    """
    # Create an adjacency list from connections
    adjacency_list = {}
    for room1, room2 in graph:
        if room1 not in adjacency_list:
            adjacency_list[room1] = []
        if room2 not in adjacency_list:
            adjacency_list[room2] = []
        adjacency_list[room1].append(room2)
        adjacency_list[room2].append(room1)  # Undirected graph

    # Queue for BFS
    queue = deque([(start, [start])])
    # Keep track of visited rooms
    visited = {start}

    while queue:
        room, path = queue.popleft()

        # If we reached the end room, return the path
        if room == end:
            return path

        # Add unvisited neighbors to the queue
        for neighbor in adjacency_list.get(room, []):
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, path + [neighbor]))

    # If no path is found
    return None

def create_networkx_graph(room_positions, connections, full_path, output_path):
    """
    Create a separate NetworkX graph visualization and save it
    """
    # Create graph
    G = nx.Graph()

    # Add nodes
    for room, pos in room_positions.items():
        # Scale down the positions for a nicer plot
        scaled_pos = (pos[0]/100, pos[1]/100)
        G.add_node(room, pos=scaled_pos)

    # Add edges
    for room1, room2 in connections:
        G.add_edge(room1, room2)

    # Create path edges list
    if full_path and full_path[0] != "Path not found":
        path_edges = [(full_path[i], full_path[i+1]) for i in range(len(full_path)-1)]
    else:
        path_edges = []

    # Get positions for drawing
    pos = nx.get_node_attributes(G, 'pos')

    # Create plot with a clean, modern style
    plt.figure(figsize=(8, 6), facecolor='white')

    # Draw edges with refined styling
    nx.draw_networkx_edges(G, pos, width=1, alpha=0.3, edge_color='#aaaaaa')
    nx.draw_networkx_edges(G, pos, edgelist=path_edges, width=2, edge_color='#ff3333')

    # Node colors based on path - smaller nodes for better readability
    node_colors = ['#ff3333' if node in full_path else '#66b3ff' for node in G.nodes()]
    nx.draw_networkx_nodes(G, pos, node_size=250, node_color=node_colors,
                          edgecolors='white', linewidths=1)

    # Cleaner, more readable labels
    label_font = {'fontname': 'Arial', 'fontsize': 8, 'fontweight': 'bold'}
    nx.draw_networkx_labels(G, pos, font_size=8, font_family="sans-serif",
                           font_color='white', font_weight='bold')

    # Create a clean legend
    red_circle = plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='#ff3333',
                           markersize=8, label='Rooms in path')
    blue_circle = plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='#66b3ff',
                            markersize=8, label='Other rooms')
    red_line = plt.Line2D([0], [0], color='#ff3333', linewidth=2, label='Path connections')
    gray_line = plt.Line2D([0], [0], color='#aaaaaa', linewidth=1, label='Other connections')

    # Add the legend with a clean style
    plt.legend(handles=[red_circle, blue_circle, red_line, gray_line],
              loc='upper right', frameon=True, facecolor='white', framealpha=0.9,
              edgecolor='#dddddd', fontsize=8)

    # Title with better styling
    plt.title("Floor Plan Graph", fontsize=12, fontweight='bold', pad=10)
    plt.axis('off')
    plt.tight_layout()

    # Add path information as a subtitle
    path_text = "Path: " + " → ".join(full_path)
    plt.figtext(0.5, 0.01, path_text, ha='center', fontsize=8,
               bbox=dict(facecolor='white', alpha=0.8, edgecolor='#dddddd', boxstyle='round,pad=0.5'))

    # Save the NetworkX graph visualization with higher DPI for sharper image
    plt.savefig(output_path, dpi=300, bbox_inches='tight')

    return G

def main():
    try:
        # Define the input and output paths
        image_path = "/content/Screenshot 2025-03-04 212424.png"  # Update with your image path
        output_path = "floor_plan_with_graph.png"
        networkx_output = "floor_plan_networkx_graph.png"
        processed_output = "floor_plan_processed.png"

        # For a cleaner visualization, enable antialiasing
        import matplotlib
        matplotlib.rcParams['figure.dpi'] = 300
        matplotlib.rcParams['savefig.dpi'] = 300
        matplotlib.rcParams['path.simplify'] = True
        matplotlib.rcParams['path.simplify_threshold'] = 1.0

        # Process the floor plan and add the graph visualization
        result_img, processed_img, path = process_floor_plan_with_graph(image_path)

        # Get room positions and connections for the NetworkX graph
        _, _, room_positions, connections, _ = process_floor_plan(image_path)

        # Save the results
        cv2.imwrite(output_path, result_img)
        cv2.imwrite(processed_output, processed_img)

        # Create and save a separate NetworkX graph visualization
        create_networkx_graph(room_positions, connections, path, networkx_output)

        # Display the final result
        plt.figure(figsize=(16, 6))

        plt.subplot(1, 2, 1)
        result_img_rgb = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
        plt.imshow(result_img_rgb)
        plt.title("Floor Plan with Path Graph")
        plt.axis('off')

        plt.subplot(1, 2, 2)
        processed_img_rgb = cv2.cvtColor(processed_img, cv2.COLOR_BGR2RGB)
        plt.imshow(processed_img_rgb)
        plt.title("Processed Floor Plan (Walls and Rooms)")
        plt.axis('off')

        plt.tight_layout()
        plt.show()

        print(f"Path from Bedroom to Living Room to Bathroom: {' → '.join(path)}")
        print(f"Results saved to {output_path}, {processed_output}, and {networkx_output}")

    except Exception as e:
        print(f"An error occurred: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

An error occurred: Could not load image from /content/Screenshot 2025-03-04 212424.png


Traceback (most recent call last):
  File "<ipython-input-8-8e65a0ac26c1>", line 456, in main
    result_img, processed_img, path = process_floor_plan_with_graph(image_path)
                                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-8-8e65a0ac26c1>", line 182, in process_floor_plan_with_graph
    img, processed_img, room_positions, connections, doorways = process_floor_plan(image_path)
                                                                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-8-8e65a0ac26c1>", line 21, in process_floor_plan
    raise FileNotFoundError(f"Could not load image from {image_path}")
FileNotFoundError: Could not load image from /content/Screenshot 2025-03-04 212424.png
